In [11]:
# libraries
import pandas as pd

In [12]:
# get datasets
train = pd.read_csv("https://raw.githubusercontent.com/jardel-nascimento/machine-learning-getting-started/main/datasets/housing-prices/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/jardel-nascimento/machine-learning-getting-started/main/datasets/housing-prices/test.csv")

In [13]:
# select the target prediction
train.dropna(axis=0,subset=['SalePrice'], inplace=True)
y = train['SalePrice']
train.drop(['SalePrice'], axis=1, inplace=True)

In [14]:
# select features
x = train.select_dtypes(exclude=['object'])
xt = test.select_dtypes(exclude=['object'])

In [32]:
# drop columns with missed values
cols_with_missing = [col for col in x.columns if xt[col].isnull().any()]
x = x.drop(cols_with_missing, axis=1)
cols_with_missing = [col for col in xt.columns if xt[col].isnull().any()]
xt = xt.drop(cols_with_missing, axis=1)

In [33]:
#print(x.shape)
#print(xt.shape)

(1459, 26)


In [34]:
# split train validation
from sklearn.model_selection import train_test_split as tts
tx, vx, ty, vy = tts(x,y, train_size=0.8, test_size=0.2, random_state=0)

In [35]:
# review data
# print description or statistics from X
# print(x.describe().T)
# print('\n\n')

# print the top few lines
# print(x.head())

In [36]:
# define the model and validation "NOW RANDOM FORESTS"
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import mean_absolute_error as mae

In [37]:
# define models
model_1 = rfr(n_estimators=50, random_state=0)
model_2 = rfr(n_estimators=100, random_state=0)
model_3 = rfr(n_estimators=100, criterion='mae', random_state=0)
model_4 = rfr(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = rfr(n_estimators=100, max_depth=7, random_state=0)

# list models
models = [model_1,model_2,model_3,model_4,model_5]

In [38]:
# create function to compare scores
def get_mae(model,tx=tx,vx=vx,ty=ty,vy=vy):
    # fit model
    model.fit(tx,ty)
    # predict
    vp = model.predict(vx)
    # define validation
    return(mae(vy, vp))

In [41]:
# compare mae for the diferent models
df_mae = []
df_models = []
for i in range(0, len(models)):
    #print(get_mae(models[i]))
    df_mae.append(get_mae(models[i]))
    df_models.append(f'model {i}')

# select max leaf nodes for minimum mae
tests = pd.DataFrame({'model': df_models, 'mae': df_mae})
great = tests.loc[tests.query('mae != 0')['mae'].idxmin()]
print(f'the great result is in {great[0]} with score {great[1]}.')

the great result is in model 1 with score 19149.20113013699.


In [40]:
# generate prediction to test data

# set new model
model = model_2

# fit new model to full train data
model.fit(x,y)

# predict test data
p = model.predict(xt)

In [42]:
# export submission csv file
output = pd.DataFrame({'Id': test.Id,
                       'SalePrice': p})
output.to_csv('submission.csv', index=False)